In [26]:
import sys
import time
import dotenv
import os
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from skmultilearn.model_selection import IterativeStratification
import json


In [27]:
sys.path.append("../..")
from src.algorithms.multi_objective.MOEA_D_AM import MOEA_D_AM

In [28]:
env = dotenv.find_dotenv()

dataset_path = '/home/adel/Documents/Code/Ant-Miner/datasets/multi_label/chd_49.csv'
results_path = dotenv.get_key(env, 'SLC_RESULTS_DIR')
archive_path = dotenv.get_key(env, 'SLC_ARCHIVE_DIR')

In [29]:
args = {
    "task": "single",
    "population": 50,
    "neighbors": 10,
    "groups": 5,
    "min_examples": 10,
    "max_uncovered": 10,
    "max_iter": 100,
    "gamma": 0.9,
    "delta": 0.05,
    "alpha": 1,
    "beta": 1,
    "p": 0.9,
    "pruning": 0,
    "decomposition": "weighted",
    "archive_type": "rules",
    "rulesets": None,
    "ruleset_size": 2,
    "prediction_strat": "all",
    "cross_val": 1,
    "folds": 5,
    "random_state": None,
    "runs": 1,
}

In [30]:
dataset = pd.read_csv(dataset_path)



In [32]:
labels = [col for col in dataset.columns if "label" in col]
X = dataset.drop(columns=labels)

preds =[]

for label in labels:
    y = dataset[label]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
    train = pd.concat([X_train, y_train], axis=1)
    train.rename(columns={label: 'class'}, inplace=True)
    test = pd.concat([X_test, y_test], axis=1)

    moea_d_aco = MOEA_D_AM(
            task = args['task'],
            population=args['population'],
            neighbors=args['neighbors'],
            groups=args['groups'],
            min_examples=args['min_examples'],
            max_uncovered=args['max_uncovered'],
            max_iter=args['max_iter'],
            p=args['p'],
            gamma=args['gamma'],
            alpha=args['alpha'],
            beta=args['beta'],
            delta=args['delta'],
            pruning=args['pruning'],
            decomposition=args['decomposition'],
            archive_type=args['archive_type'],
            rulesets=args['rulesets'],
            random_state=args['random_state']
        )
    moea_d_aco.run(train, ['class'])
    preds.append(moea_d_aco.predict(test, ['class']))

df = pd.concat(preds, axis=1)
df.columns = labels



Running MOEA/D-AM: 100%|██████████| 100/100 [00:29<00:00,  3.37it/s]

[]


ValueError: No rules found in the archive. Run the algorithm first.